In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Connect4 AI Assignment\n",
    "\n",
    "**Team Members:**\n",
    "- Team Member 1\n",
    "- Team Member 2\n",
    "- Team Member 3\n",
    "\n",
    "## Introduction\n",
    "\n",
    "This notebook implements and evaluates two AI algorithms for playing the Connect4 game:\n",
    "\n",
    "1. Monte Carlo Tree Search (MCTS) with UCT\n",
    "2. ID3 Decision Tree trained on MCTS-generated data\n",
    "\n",
    "We will evaluate their performance and compare their strengths and weaknesses.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import necessary libraries\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import os\n",
    "import sys\n",
    "import time\n",
    "from IPython.display import display, HTML\n",
    "\n",
    "# Add the project root to the path for imports\n",
    "sys.path.append(os.path.dirname(os.getcwd()))\n",
    "\n",
    "# Import project modules\n",
    "from game_structure import style as s\n",
    "from game_structure import game_engine as game\n",
    "from ai_alg.monte_carlo import monte_carlo, Node\n",
    "from ai_alg.alpha_beta import alpha_beta\n",
    "from ai_alg.ID3 import ID3Tree"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Game Implementation\n",
    "\n",
    "First, let's review the Connect4 game structure implementation and demonstrate how it works."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Helper function to display Connect4 board in the notebook\n",
    "def display_board(board):\n",
    "    \"\"\"Display a Connect4 board in the notebook\"\"\"\n",
    "    rows, cols = board.shape\n",
    "    html = '<table style=\"border-collapse: collapse; border: 3px solid blue; width: 280px; height: 240px;\">' \n",
    "    \n",
    "    for r in range(rows-1, -1, -1):  # Display from top to bottom\n",
    "        html += '<tr>'\n",
    "        for c in range(cols):\n",
    "            color = 'white'\n",
    "            if board[r, c] == 1:\n",
    "                color = 'red'\n",
    "            elif board[r, c] == 2:\n",
    "                color = 'yellow'\n",
    "            html += f'<td style=\"border: 1px solid black; width: 40px; height: 40px; \\\n",
    "                      background-color: {color}; border-radius: 50%;\"></td>'\n",
    "        html += '</tr>'\n",
    "    \n",
    "    # Add column numbers\n",
    "    html += '<tr>'\n",
    "    for c in range(cols):\n",
    "        html += f'<td style=\"text-align: center; font-weight: bold;\">{c}</td>'\n",
    "    html += '</tr>'\n",
    "    \n",
    "    html += '</table>'\n",
    "    return HTML(html)\n",
    "\n",
    "# Create an empty board\n",
    "board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)\n",
    "\n",
    "# Make some example moves\n",
    "game.drop_piece(board, 0, 3, 1)  # Player 1 in column 3, row 0\n",
    "game.drop_piece(board, 0, 2, 2)  # Player 2 in column 2, row 0\n",
    "game.drop_piece(board, 0, 4, 1)  # Player 1 in column 4, row 0\n",
    "game.drop_piece(board, 1, 3, 2)  # Player 2 in column 3, row 1\n",
    "\n",
    "# Display the board\n",
    "display_board(board)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Monte Carlo Tree Search Implementation\n",
    "\n",
    "Now let's examine our MCTS implementation and demonstrate how it selects moves."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create board with a potential winning move\n",
    "board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)\n",
    "\n",
    "# Set up a position where Player 2 can win with a move\n",
    "for col, row in [(2, 0), (3, 0), (4, 0)]:\n",
    "    game.drop_piece(board, row, col, 2)\n",
    "\n",
    "display_board(board)\n",
    "print(\"Player 2 can win by playing column 1 or 5\")\n",
    "\n",
    "# Let's run MCTS and see what it suggests\n",
    "root = Node(board=board, last_player=s.FIRST_PLAYER_PIECE)\n",
    "mc = monte_carlo(root)\n",
    "start_time = time.time()\n",
    "move = mc.start(2)  # Run MCTS for 2 seconds\n",
    "elapsed = time.time() - start_time\n",
    "\n",
    "print(f\"MCTS selected column: {move} in {elapsed:.2f} seconds\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### MCTS Exploration Parameter Analysis\n",
    "\n",
    "Let's analyze how the number of children explored affects MCTS performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Let's analyze different numbers of selected children\n",
    "# In our monte_carlo.py, the select_children method limits exploration to 4 children\n",
    "# Let's modify it temporarily for testing\n",
    "\n",
    "def test_child_selection(board, max_children_values=[2, 4, 7], search_time=1.0):\n",
    "    results = []\n",
    "    \n",
    "    for max_children in max_children_values:\n",
    "        # Temporarily modify Node.select_children to use our max_children parameter\n",
    "        original_select_children = Node.select_children\n",
    "        \n",
    "        def modified_select_children(self):\n",
    "            if len(self.children) > max_children:\n",
    "                return random.sample(self.children, max_children)\n",
    "            return self.children\n",
    "        \n",
    "        # Apply the monkey patch\n",
    "        Node.select_children = modified_select_children\n",
    "        \n",
    "        # Run MCTS with this configuration\n",
    "        root = Node(board=board.copy(), last_player=s.FIRST_PLAYER_PIECE)\n",
    "        mc = monte_carlo(root)\n",
    "        start_time = time.time()\n",
    "        move = mc.start(search_time)\n",
    "        elapsed = time.time() - start_time\n",
    "        \n",
    "        # Calculate number of nodes explored\n",
    "        nodes_explored = count_nodes(root)\n",
    "        \n",
    "        results.append({\n",
    "            'max_children': max_children,\n",
    "            'move_selected': move,\n",
    "            'time': elapsed,\n",
    "            'nodes_explored': nodes_explored\n",
    "        })\n",
    "        \n",
    "        # Restore original method\n",
    "        Node.select_children = original_select_children\n",
    "    \n",
    "    return pd.DataFrame(results)\n",
    "\n",
    "# Helper to count nodes in the tree\n",
    "def count_nodes(node):\n",
    "    if not node.children:\n",
    "        return 1\n",
    "    return 1 + sum(count_nodes(child[0]) for child in node.children)\n",
    "\n",
    "# Test with our board\n",
    "import random  # For random.sample\n",
    "results_df = test_child_selection(board)\n",
    "results_df"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Let's visualize the results\n",
    "plt.figure(figsize=(12, 5))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.bar(results_df['max_children'].astype(str), results_df['nodes_explored'])\n",
    "plt.title('Number of Nodes Explored')\n",
    "plt.xlabel('Max Children per Node')\n",
    "plt.ylabel('Nodes Explored')\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.bar(results_df['max_children'].astype(str), results_df['time'])\n",
    "plt.title('Computation Time')\n",
    "plt.xlabel('Max Children per Node')\n",
    "plt.ylabel('Time (seconds)')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Discussion of results\n",
    "print(\"Results Analysis:\")\n",
    "print(\"1. As we increase the number of children explored, we see a significant increase in nodes explored\")\n",
    "print(\"2. This leads to more thorough search but higher computational costs\")\n",
    "print(\"3. The optimal value balances exploration breadth with computational efficiency\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Dataset Generation\n",
    "\n",
    "Now, let's generate a dataset of Connect4 states and moves using MCTS. For demonstration purposes, we'll generate a small dataset in the notebook."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Generate a small dataset for demonstration\n",
    "def generate_demo_dataset(n_games=5, search_time=1.0):\n",
    "    all_records = []\n",
    "    \n",
    "    for game_idx in range(n_games):\n",
    "        print(f\"Generating game {game_idx + 1}/{n_games}...\")\n",
    "        board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)\n",
    "        game_records = []\n",
    "        turn = s.FIRST_PLAYER_PIECE\n",
    "        \n",
    "        while True:\n",
    "            # Record current state\n",
    "            state = board.flatten().tolist()\n",
    "            \n",
    "            # Get move from MCTS\n",
    "            last_player = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE\n",
    "            root = Node(board=board.copy(), last_player=last_player)\n",
    "            mc = monte_carlo(root)\n",
    "            move = mc.start(search_time)\n",
    "            \n",
    "            # Record state and chosen move\n",
    "            game_records.append(state + [move])\n",
    "            \n",
    "            # Make the move\n",
    "            row = game.get_next_open_row(board, move)\n",
    "            game.drop_piece(board, row, move, turn)\n",
    "            \n",
    "            # Check if game is over\n",
    "            if game.winning_move(board, turn) or game.is_game_tied(board):\n",
    "                print(f\"Game over after {len(game_records)} moves\")\n",
    "                print(\"Final board:\")\n",
    "                display(display_board(board))\n",
    "                break\n",
    "            \n",
    "            # Switch players\n",
    "            turn = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE\n",
    "        \n",
    "        all_records.extend(game_records)\n",
    "    \n",
    "    # Create DataFrame\n",
    "    columns = [f\"cell_{i}\" for i in range(42)] + [\"move\"]\n",
    "    df = pd.DataFrame(all_records, columns=columns)\n",
    "    return df\n",
    "\n",
    "# Generate a small demo dataset\n",
    "demo_dataset = generate_demo_dataset(n_games=2, search_time=0.5)\n",
    "print(f\"Generated dataset with {len(demo_dataset)} state-move pairs\")\n",
    "demo_dataset.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Load Full Dataset\n",
    "\n",
    "In practice, we would generate a larger dataset using our script and load it here."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Check if the full dataset exists, otherwise use our demo dataset\n",
    "dataset_path = os.path.join(\"data\", \"connect4_dataset.csv\")\n",
    "if os.path.exists(dataset_path):\n",
    "    print(f\"Loading dataset from {dataset_path}\")\n",
    "    dataset = pd.read_csv(dataset_path)\n",
    "else:\n",
    "    print(\"Full dataset not found, using demo dataset\")\n",
    "    dataset = demo_dataset\n",
    "    \n",
    "print(f\"Dataset shape: {dataset.shape}\")\n",
    "\n",
    "# Analyze move distribution\n",
    "plt.figure(figsize=(10, 5))\n",
    "move_counts = dataset['move'].value_counts().sort_index()\n",
    "plt.bar(move_counts.index, move_counts.values)\n",
    "plt.title('Distribution of Moves in Dataset')\n",
    "plt.xlabel('Column')\n",
    "plt.ylabel('Count')\n",
    "plt.xticks(range(7))\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "print(\"Move distribution (%):\\n\")\n",
    "move_percentage = move_counts / len(dataset) * 100\n",
    "for col, pct in move_percentage.items():\n",
    "    print(f\"Column {col}: {pct:.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. ID3 Decision Tree Implementation and Training\n",
    "\n",
    "Now, let's train our ID3 decision tree on the Connect4 dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Split dataset into training and testing sets\n",
    "def manual_train_test_split(X, y, test_size=0.2, random_seed=42):\n",
    "    \"\"\"Split arrays into random train and test subsets\"\"\"\n",
    "    np.random.seed(random_seed)\n",
    "    indices = np.random.permutation(len(X))\n",
    "    test_size = int(len(X) * test_size)\n",
    "    test_indices = indices[:test_size]\n",
    "    train_indices = indices[test_size:]\n",
    "    \n",
    "    X_train = X.iloc[train_indices] if isinstance(X, pd.DataFrame) else X[train_indices]\n",
    "    X_test = X.iloc[test_indices] if isinstance(X, pd.DataFrame) else X[test_indices]\n",
    "    y_train = y.iloc[train_indices] if isinstance(y, pd.Series) else y[train_indices]\n",
    "    y_test = y.iloc[test_indices] if isinstance(y, pd.Series) else y[test_indices]\n",
    "    \n",
    "    return X_train, X_test, y_train, y_test\n",
    "\n",
    "# Prepare the data\n",
    "X = dataset.iloc[:, :-1]\n",
    "y = dataset.iloc[:, -1]\n",
    "\n",
    "X_train, X_test, y_train, y_test = manual_train_test_split(X, y, test_size=0.2)\n",
    "print(f\"Training set size: {len(X_train)}\")\n",
    "print(f\"Testing set size: {len(X_test)}\")\n",
    "\n",
    "# Train the ID3 decision tree\n",
    "print(\"Training ID3 decision tree...\")\n",
    "start_time = time.time()\n",
    "id3_tree = ID3Tree(max_depth=10)  # Limiting depth for reasonable training time\n",
    "id3_tree.fit(X_train, y_train)\n",
    "print(f\"Training completed in {time.time() - start_time:.2f} seconds\")\n",
    "\n",
    "# Evaluate the model\n",
    "y_pred = id3_tree.predict(X_test)\n",
    "accuracy = np.mean(y_pred == y_test) * 100\n",
    "print(f\"ID3 accuracy on test set: {accuracy:.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Visualize Decision Tree Structure\n",
    "\n",
    "Let's examine a simplified version of the learned decision tree."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Helper function to visualize a simplified version of the tree\n",
    "def visualize_tree(tree, max_depth=3, depth=0, prefix=\"\"):\n",
    "    \"\"\"Print a simplified visualization of the decision tree\"\"\"\n",
    "    if depth > max_depth:\n",
    "        return \"...\"  # Too deep, truncate\n",
    "    \n",
    "    if not isinstance(tree, dict):\n",
    "        return f\"→ Column {tree}\"  # Leaf node (predicted move)\n",
    "    \n",
    "    # Get the attribute and its value\n",
    "    attr, branches = list(tree.items())[0]\n",
    "    feature, value = attr\n",
    "    \n",
    "    # Convert feature name to board position for better interpretation\n",
    "    feature_idx = int(feature.replace(\"cell_\", \"\"))\n",
    "    row = feature_idx // s.COLUMNS\n",
    "    col = feature_idx % s.COLUMNS\n",
    "    \n",
    "    result = f\"If cell({row},{col}) == {value}:\\n\"\n",
    "    result += f\"{prefix}├─ Yes: {visualize_tree(branches['left'], max_depth, depth+1, prefix+'│  ')}\\n\"\n",
    "    result += f\"{prefix}└─ No: {visualize_tree(branches['right'], max_depth, depth+1, prefix+'   ')}\"\n",
    "    return result\n",
    "\n",
    "# Visualize the tree\n",
    "print(\"Simplified Decision Tree Visualization (limited to depth 3):\")\n",
    "print(visualize_tree(id3_tree.tree, max_depth=3))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Compare Algorithm Performance\n",
    "\n",
    "Let's compare the performance of our different algorithms: MCTS, Alpha-Beta, and ID3."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def compare_algorithms(board, time_limit=1.0):\n",
    "    \"\"\"Compare different algorithms on the same board position\"\"\"\n",
    "    results = {}\n",
    "    \n",
    "    # 1. Monte Carlo Tree Search\n",
    "    start_time = time.time()\n",
    "    root = Node(board=board.copy(), last_player=s.FIRST_PLAYER_PIECE)\n",
    "    mc = monte_carlo(root)\n",
    "    mcts_move = mc.start(time_limit)\n",
    "    mcts_time = time.time() - start_time\n",
    "    results['MCTS'] = {'move': mcts_move, 'time': mcts_time}\n",
    "    \n",
    "    # 2. Alpha-Beta Pruning\n",
    "    start_time = time.time()\n",
    "    ab_move = alpha_beta(board.copy())\n",
    "    ab_time = time.time() - start_time\n",
    "    results['Alpha-Beta'] = {'move': ab_move, 'time': ab_time}\n",
    "    \n",
    "    # 3. ID3 Decision Tree\n",
    "    start_time = time.time()\n",
    "    # Convert board to feature vector\n",
    "    features = pd.DataFrame([board.flatten().tolist()], columns=[f'cell_{i}' for i in range(42)])\n",
    "    id3_move = id3_tree.predict(features).iloc[0]\n",
    "    id3_time = time.time() - start_time\n",
    "    results['ID3'] = {'move': id3_move, 'time': id3_time}\n",
    "    \n",
    "    return results\n",
    "\n",
    "# Create a few test positions\n",
    "test_positions = []\n",
    "\n",
    "# Position 1: Empty board\n",
    "board1 = np.zeros((s.ROWS, s.COLUMNS), dtype=int)\n",
    "test_positions.append((\"Empty board\", board1))\n",
    "\n",
    "# Position 2: Mid-game position\n",
    "board2 = np.zeros((s.ROWS, s.COLUMNS), dtype=int)\n",
    "moves = [(0, 3, 1), (0, 2, 2), (0, 4, 1), (0, 1, 2), (1, 3, 1), (0, 0, 2)]\n",
    "for r, c, p in moves:\n",
    "    game.drop_piece(board2, r, c, p)\n",
    "test_positions.append((\"Mid-game position\", board2))\n",
    "\n",
    "# Position 3: Near-win position\n",
    "board3 = np.zeros((s.ROWS, s.COLUMNS), dtype=int)\n",
    "moves = [(0, 0, 1), (0, 1, 1), (0, 2, 1), (0, 6, 2), (1, 6, 2), (2, 6, 2)]\n",
    "for r, c, p in moves:\n",
    "    game.drop_piece(board3, r, c, p)\n",
    "test_positions.append((\"Near-win position\", board3))\n",
    "\n",
    "# Compare algorithms on each position\n",
    "all_results = []\n",
    "\n",
    "for name, board in test_positions:\n",
    "    print(f\"\\nPosition: {name}\")\n",
    "    display(display_board(board))\n",
    "    \n",
    "    results = compare_algorithms(board)\n",
    "    \n",
    "    print(\"Algorithm comparison:\")\n",
    "    for algo, data in results.items():\n",
    "        print(f\"{algo}: chose column {data['move']} in {data['time']*1000:.2f} ms\")\n",
    "    \n",
    "    # Add to results for later analysis\n",
    "    for algo, data in results.items():\n",
    "        all_results.append({\n",
    "            'position': name,\n",
    "            'algorithm': algo,\n",
    "            'move': data['move'],\n",
    "            'time_ms': data['time'] * 1000\n",
    "        })\n",
    "\n",
    "# Convert to DataFrame\n",
    "results_df = pd.DataFrame(all_results)\n",
    "\n",
    "# Visualize time comparison\n",
    "plt.figure(figsize=(10, 6))\n",
    "positions = results_df['position'].unique()\n",
    "algorithms = results_df['algorithm'].unique()\n",
    "x = np.arange(len(positions))\n",
    "width = 0.25\n",
    "\n",
    "for i, algo in enumerate(algorithms):\n",
    "    algo_data = results_df[results_df['algorithm'] == algo]\n",
    "    plt.bar(x + i*width - width, algo_data['time_ms'], width, label=algo)\n",
    "\n",
    "plt.title('Algorithm Performance Comparison')\n",
    "plt.xlabel('Position')\n",
    "plt.ylabel('Time (ms)')\n",
    "plt.xticks(x, positions)\n",
    "plt.legend()\n",
    "plt.yscale('log')  # Log scale to show large differences\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Discussion and Conclusion\n",
    "\n",
    "### Algorithm Comparison\n",
    "\n",
    "Let's summarize what we've learned about each algorithm:\n",
    "\n",
    "#### Monte Carlo Tree Search (MCTS)\n",
    "- **Strengths**: Explores the game tree effectively, doesn't require a specific heuristic evaluation function, and can be tuned by adjusting exploration parameters.\n",
    "- **Weaknesses**: Computationally expensive, performance depends on available search time.\n",
    "- **Best use case**: When high-quality play is needed and computational resources are available.\n",
    "\n",
    "#### Alpha-Beta Pruning\n",
    "- **Strengths**: Can search deeper than MCTS in the same time budget by pruning unpromising branches.\n",
    "- **Weaknesses**: Heavily dependent on the quality of the evaluation function.\n",
    "- **Best use case**: When a good evaluation function is available and consistent decision-making is important.\n",
    "\n",
    "#### ID3 Decision Tree\n",
    "- **Strengths**: Extremely fast execution time, learns patterns from training data automatically.\n",
    "- **Weaknesses**: Quality entirely depends on training data, doesn't adapt to new situations.\n",
    "- **Best use case**: When execution speed is critical or as part of a hybrid system.\n",
    "\n",
    "### Practical Recommendations\n",
    "\n",
    "Based on our experiments, we recommend:\n",
    "\n",
    "1. Use ID3 for real-time play where decisions must be made quickly\n",
    "2. Use MCTS for higher quality play when computation time is available\n",
    "3. Consider a hybrid approach: use ID3 for opening moves and early/mid-game, then switch to MCTS for critical endgame situations\n",
    "\n",
    "### Future Work\n",
    "\n",
    "Potential improvements for future development:\n",
    "\n",
    "1. Implement a hybrid algorithm that combines the strengths of multiple approaches\n",
    "2. Improve the training dataset quality by using stronger MCTS parameters\n",
    "3. Explore feature engineering to improve the ID3 model's performance\n",
    "4. Implement a neural network approach as an alternative learning method"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Running a Full Connect4 Game\n",
    "\n",
    "For completeness, let's run a full game between two AI algorithms."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def run_ai_vs_ai_game(player1_algo='MCTS', player2_algo='ID3', search_time=1.0):\n",
    "    \"\"\"Run a game between two AI algorithms\"\"\"\n",
    "    board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)\n",
    "    moves_history = []\n",
    "    \n",
    "    # Current player (1 or 2)\n",
    "    turn = s.FIRST_PLAYER_PIECE\n",
    "    \n",
    "    # Map algorithm names to functions\n",
    "    algo_funcs = {\n",
    "        'MCTS': lambda b: monte_carlo(Node(b.copy(), s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE)).start(search_time),\n",
    "        'Alpha-Beta': lambda b: alpha_beta(b.copy()),\n",
    "        'ID3': lambda b: id3_tree.predict(pd.DataFrame([b.flatten().tolist()], columns=[f'cell_{i}' for i in range(42)])).iloc[0]\n",
    "    }\n",
    "    \n",
    "    print(f\"Starting game: {player1_algo} (Player 1) vs {player2_algo} (Player 2)\")\n",
    "    display(display_board(board))\n",
    "    \n",
    "    # Game loop\n",
    "    while True:\n",
    "        # Determine current algorithm\n",
    "        current_algo = player1_algo if turn == s.FIRST_PLAYER_PIECE else player2_algo\n",
    "        \n",
    "        # Get move from appropriate algorithm\n",
    "        start_time = time.time()\n",
    "        move = algo_funcs[current_algo](board)\n",
    "        elapsed = time.time() - start_time\n",
    "        \n",
    "        print(f\"Player {turn} ({current_algo}) chose column {move} in {elapsed*1000:.2f} ms\")\n",
    "        \n",
    "        # Make the move\n",
    "        row = game.get_next_open_row(board, move)\n",
    "        game.drop_piece(board, row, move, turn)\n",
    "        moves_history.append((turn, move))\n",
    "        \n",
    "        # Display updated board\n",
    "        display(display_board(board))\n",
    "        \n",
    "        # Check if game is over\n",
    "        if game.winning_move(board, turn):\n",
    "            print(f\"Player {turn} ({current_algo}) wins after {len(moves_history)} moves!\")\n",
    "            break\n",
    "        \n",
    "        if game.is_game_tied(board):\n",
    "            print(f\"Game is tied after {len(moves_history)} moves!\")\n",
    "            break\n",
    "        \n",
    "        # Switch players\n",
    "        turn = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE\n",
    "    \n",
    "    return board, moves_history\n",
    "\n",
    "# Run a game between MCTS and ID3\n",
    "final_board, moves = run_ai_vs_ai_game(player1_algo='MCTS', player2_algo='ID3', search_time=0.5)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}